## Run this cell to set up paths, imports, and W&B logging

In [ ]:
%cd ..

import os
from PIL import Image

import wandb
import numpy as np
import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import src.model
from src.training import train, validate

## 1. Initialize W&B project

In [ ]:
wandb.init(project="iml", config={
    "learning_rate": 0.001,
    "epochs": 80,
    "batch_size": 32,
    "image_size": (32, 32) # TODO: choose best image_size
})
config = wandb.config

# Configure directories
TRAIN_DIR = "./datasets/train/"
VAL_DIR = "./datasets/val/"
TEST_DIR = "./datasets/test/"

# Speakers of such labels should be admitted to enter
VALID_ACCESS_LABELS = {
    "f1", "f7", "f8", "m3", "m6", "m8"
}

## 2. Define a custom dataset class to load spectrograms from files

In [ ]:
class SpectrogramDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = self.files[idx]
        speaker_id = img_path.split('/')[-1].split('_')[0]
        label = int(speaker_id in VALID_ACCESS_LABELS)
        image = Image.open(img_path).convert("L")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


## 3. Set up data transformations and data loaders

In [ ]:
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    # TODO: normalization
])

train_dataset = SpectrogramDataset(TRAIN_DIR, transform=transform)
val_dataset = SpectrogramDataset(VAL_DIR, transform=transform)
test_dataset = SpectrogramDataset(TEST_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)


## 4. Initialize model, optimizer, and set device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = src.model.SimpleCNN().to(device)
model = src.model.TutorialCNN() # - for 32x32 images
model.device = device
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)


## 5. Training and validation loop

In [ ]:

for epoch in range(config.epochs):
    print(f"Epoch {epoch+1}/{config.epochs}")
    
    train(model, train_loader, criterion, optimizer, epoch, wandb.log, len(train_loader) // 5 - 1)
    recall, precision, val_f1 = validate(model, val_loader)
    wandb.log({"validation/recall": recall, "validation/precision": precision, "validation/f1": val_f1, "epoch": epoch+1})
    print("Recall: ", recall, ", Precision: ", precision, ", Validation F1: ", val_f1)


## 6. Save the model

In [ ]:

model_path = "./models/simple_cnn.pth"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
torch.save(model.state_dict(), model_path)
wandb.save(model_path)
wandb.finish()

print("Training complete and model saved!")
